# 인터파크 일일 베스트셀러 책 순위 크롤링

In [89]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [90]:
site = 'http://book.interpark.com'         
url = f'{site}/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001#'
req = requests.get(url)
html = req.text
#html

In [91]:
soup = BeautifulSoup(html, 'html.parser')

In [92]:
lis = soup.select('div.rankBestContentList > ol > li')
len(lis)

15

In [145]:
li = lis[0]

### 제목

In [146]:
title = li.select_one('div.itemName').get_text().strip()
title

'트렌드 코리아 2022'

### 가격(정수)

In [147]:
price = li.select_one('span.price > em').get_text().strip()
price = int(price.replace(',',''))
price

16200

### 작가

In [148]:
author = li.select_one('span.author').get_text()
author

'김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영'

### 출판사

In [149]:
company = li.select_one('span.company').get_text()
company

'미래의창'

### 상세페이지 url

In [97]:
li = lis[0]
href = li.find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [98]:
sub_url = site + href
sub_url

'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [99]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [150]:
all_lis = sub_soup.select_one('ul.bInfo_txt').find_all()
# all_lis

### 발행일

In [131]:
date_li = all_lis[-3].string.split()
published_date = date_li[2:]
' '.join(published_date)

'2021년 10월 06일'

### 쪽수(정수)

In [140]:
page_li = all_lis[-2].string.split()
pages = page_li[-1]
int(pages)

452

### DataFrame 만들기

In [152]:
lines = []
for li in lis:
    href = li.find('a')['href']
    title = li.select_one('div.itemName').get_text().strip()
    author = li.select_one('span.author').get_text()
    company = li.select_one('span.company').get_text()
    price = li.select_one('span.price > em').get_text().strip()
    price = int(price.replace(',',''))

    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')
    all_lis = sub_soup.select_one('ul.bInfo_txt').find_all()
    date_li = all_lis[-3].string.split()
    published_date = date_li[2:]
    ' '.join(published_date)
    page_li = all_lis[-2].string.split()
    pages = page_li[-1]
    int(pages)
    d = {'제목':title, '저자':author, '출판사':company, '가격':price, '발행일':published_date, '쪽수':pages}
    lines.append(d)

df = pd.DataFrame(lines)
df

,제목,저자,출판사,가격,발행일,쪽수
0,트렌드 코리아 2022,"김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영",미래의창,16200,"[2021년, 10월, 06일]",452
1,2022 현직교사들이 들려주는 면접레시피,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,42300,"[2021년, 10월, 06일]",452
2,거인의 포트폴리오,강환국,페이지2북스,16200,"[2021년, 10월, 06일]",452
3,거꾸로 읽는 세계사,유시민(柳時敏),돌베개,15750,"[2021년, 10월, 06일]",452
4,아직 오지 않은 날들을 위하여,파스칼 브뤼크네르/이세진 역,인플루엔셜,14400,"[2021년, 10월, 06일]",452
5,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),소윤,북로망스,13500,"[2021년, 10월, 06일]",452
6,투자는 디테일에 있다,김정환,부케이,16200,"[2021년, 10월, 06일]",452
7,웰씽킹(Wealthinking),켈리 최,다산북스,14400,"[2021년, 10월, 06일]",452
8,오십에 읽는 논어,최종엽,유노북스,14400,"[2021년, 10월, 06일]",452
9,소크라테스 익스프레스(20만 부 기념 윈터 에디션),에릭 와이너/김하현 역,어크로스,16200,"[2021년, 10월, 06일]",452
